In [ ]:
# Bibliotecas
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import musdb
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import gc
import time
import IPython.display as ipd
import random 
import museval

%matplotlib inline 

In [ ]:
# Importação do banco de dados e pré-processamento
#RODAR SÓ SE NÃO TIVER SALVO
mus = musdb.DB(root="/Users/gusta/MUSDB18")
#Stereo
# 0-99 Train set
# 100-149 Test set
# 0 - The mixture,
# 1 - The drums,
# 2 - The bass,
# 3 - The rest of the accompaniment,
# 4 - The vocals.

#Conversão de stereo para mono e reamostragem
sr = 16000
i = 0 

for tracks in mus:
    aux = librosa.to_mono(tracks.stems[0,:,:].T)
    aux = librosa.resample(aux, orig_sr = 44100, target_sr = sr, res_type='kaiser_best', fix=True, scale=False)
    
    aux_per = librosa.to_mono(tracks.stems[1,:,:].T)
    aux_per = librosa.resample(aux_per, orig_sr = 44100, target_sr = sr, res_type='kaiser_best', fix=True, scale=False)
    
    aux_har = librosa.to_mono(tracks.stems[3,:,:].T)
    aux_har = librosa.resample(aux_har, orig_sr = 44100, target_sr = sr, res_type='kaiser_best', fix=True, scale=False)
    
    
    df = pd.DataFrame(aux)
    df.to_csv('C:/Users/gusta\Misturas/'+'Mixture_'+str(i)+'.csv', index = False)
    
    df = pd.DataFrame(aux_per)
    df.to_csv('C:/Users/gusta\Percurssao/'+'Percurssao_'+str(i)+'.csv', index = False)
    
    df = pd.DataFrame(aux_har)
    df.to_csv('C:/Users/gusta\Harmonico/'+'Harmonico_'+str(i)+'.csv', index = False)
    
    i = i + 1
    if i%15 == 0:
        print(str(i/1.5)+'%')

In [ ]:
#Funções

#Function that shows specifc Spectrogram
def stem_espectrograma(music_index, stem, sr, hop_length, n_window, n_fft):
    mus = musdb.DB(root="/Users/gusta/MUSDB18")
    sr = 16000
    i = 0 

    aux = librosa.to_mono(mus.tracks[int(music_index)].stems[int(stem),:,:].T)
    aux = librosa.resample(aux, orig_sr = 44100, target_sr = sr, res_type='kaiser_best', fix=True, scale=False)

    X = librosa.stft(aux, win_length = n_window, n_fft = n_fft, hop_length = hop_length, window = 'hamming')
    D = librosa.amplitude_to_db(np.abs(X), ref=np.max)
    plt.figure(figsize=(20, 7))
    librosa.display.specshow(D, y_axis = 'linear', x_axis='time', sr = 16000,
    hop_length=hop_length) # , cmap='gray_r'
    plt.colorbar(format='%+2.0f dB')
    plt.title('Linear-frequency power spectrogram')
    
    
#Function that shows specifc Spectrogram
def mostra_espectrograma(tipo_track, indice_musica, hop_length, n_window, n_fft):
    if tipo_track == 0:
        aux = pd.read_csv('C:/Users/gusta\Misturas/'+'Mixture_'+str(indice_musica)+'.csv')
        aux = aux.to_numpy()
        aux = aux.T
    elif tipo_track == 1:
        aux = pd.read_csv('C:/Users/gusta\Percurssao/'+'Percurssao_'+str(indice_musica)+'.csv')
        aux = aux.to_numpy()
        aux = aux.T
    else:
        aux = pd.read_csv('C:/Users/gusta\Harmonico/'+'Harmonico_'+str(indice_musica)+'.csv')
        aux = aux.to_numpy()
        aux = aux.T

    X = librosa.stft(aux[0], win_length = n_window, n_fft = n_fft, hop_length = hop_length, window = 'hamming')
    D = librosa.amplitude_to_db(np.abs(X), ref=np.max)
    plt.figure(figsize=(20, 7))
    librosa.display.specshow(D, y_axis = 'linear', x_axis='time', sr = 16000,
    hop_length=hop_length) # , cmap='gray_r'
    plt.colorbar(format='%+2.0f dB')
    plt.title('Linear-frequency power spectrogram')
    
#Function that returns specifc Spectrogram
def fornece_espectrograma(tipo_track, indice_musica, hop_length, n_window, n_fft):
    #start_carregamento = time.time()
    if tipo_track == 0:
        aux = pd.read_csv('C:/Users/gusta\Misturas/'+'Mixture_'+str(indice_musica)+'.csv')
        aux = aux.to_numpy()
        aux = aux.T
    elif tipo_track == 1:
        aux = pd.read_csv('C:/Users/gusta\Percurssao/'+'Percurssao_'+str(indice_musica)+'.csv')
        aux = aux.to_numpy()
        aux = aux.T
    else:
        aux = pd.read_csv('C:/Users/gusta\Harmonico/'+'Harmonico_'+str(indice_musica)+'.csv')
        aux = aux.to_numpy()
        aux = aux.T
    #stop_carregamento = time.time()    
    #start_stft = time.time()
    X = librosa.stft(aux[0], win_length = n_window, n_fft = n_fft, hop_length = hop_length, window = 'hamming')
    #stop_stft = time.time()
    
    #print (f'Carregamento [{stop_carregamento-start_carregamento}s], STFT [{stop_stft-start_stft}s]')

    return X

#Storing Spectrogram as 32bit float 
def prepara_dataset(hop_length, n_window, n_fft): 
    for i in range(150):
        aux = pd.read_csv('C:/Users/gusta/Misturas/'+'Mixture_'+str(i)+'.csv')
        aux = aux.to_numpy()
        aux = aux.T
        aux = abs(librosa.stft(aux[0], win_length = n_window, n_fft = n_fft, hop_length = hop_length, window = 'hamming', center=False)).astype('float32')
        df = pd.DataFrame(aux)
        df.to_csv('C:/Users/gusta/Misturas/Espectros/'+'Mixture_'+str(i)+'.csv', index = False)

        aux = pd.read_csv('C:/Users/gusta/Percurssao/'+'Percurssao_'+str(i)+'.csv')
        aux = aux.to_numpy()
        aux = aux.T
        aux = abs(librosa.stft(aux[0], win_length = n_window, n_fft = n_fft, hop_length = hop_length, window = 'hamming', center=False)).astype('float32')        
        df = pd.DataFrame(aux)
        df.to_csv('C:/Users/gusta/Percurssao/Espectros/'+'Percurssao_'+str(i)+'.csv', index = False)

        aux = pd.read_csv('C:/Users/gusta/Harmonico/'+'Harmonico_'+str(i)+'.csv')
        aux = aux.to_numpy()
        aux = aux.T
        aux = abs(librosa.stft(aux[0], win_length = n_window, n_fft = n_fft, hop_length = hop_length, window = 'hamming', center=False)).astype('float32')        
        df = pd.DataFrame(aux)
        df.to_csv('C:/Users/gusta/Harmonico/Espectros/'+'Harmonico_'+str(i)+'.csv', index = False)
        i = i + 1
        print(i)
        if i%15 == 0:
            print(str(i/1.5)+'%')

#Load Spectrogram in Tensors 
def carrega_dataset(dataset_type, n_tracks, hop_length, n_window, n_fft): 
    size_bytes = 0
    esp_sizes = np.zeros(n_tracks)
    esp_hops = np.zeros(n_tracks)
    
    if dataset_type == 'train':
        for i in range(n_tracks):
            x_mix = pd.read_csv('C:/Users/gusta/Misturas/Espectros/'+'Mixture_'+str(i)+'.csv')
            x_mix = x_mix.to_numpy()
            size_bytes = size_bytes + x_mix.nbytes
            esp_sizes[i] = x_mix.shape[1]
            if i > 0:
               esp_hops[i] = esp_hops[i-1]+esp_sizes[i-1]
        print("Dataset: "+str(dataset_type)+" Tamanho estimado: "+str(3*size_bytes/(1024**3))+" GB")

        dataset_mix = torch.zeros(int(np.floor((n_fft/2 +1))),int(esp_hops[n_tracks-1]+esp_sizes[n_tracks-1]))
        dataset_per = torch.zeros(int(np.floor((n_fft/2 +1))),int(esp_hops[n_tracks-1]+esp_sizes[n_tracks-1]))
        dataset_har = torch.zeros(int(np.floor((n_fft/2 +1))),int(esp_hops[n_tracks-1]+esp_sizes[n_tracks-1]))

        for i in range(n_tracks):
            esp_mix = pd.read_csv('C:/Users/gusta/Misturas/Espectros/'+'Mixture_'+str(i)+'.csv') 
            esp_mix = esp_mix.to_numpy()

            esp_per = pd.read_csv('C:/Users/gusta/Percurssao/Espectros/'+'Percurssao_'+str(i)+'.csv') 
            esp_per = esp_per.to_numpy()

            esp_har = pd.read_csv('C:/Users/gusta/Harmonico/Espectros/'+'Harmonico_'+str(i)+'.csv') 
            esp_har = esp_har.to_numpy()

            dataset_mix[:,int(esp_hops[i]):int(esp_hops[i]+esp_sizes[i])] = torch.from_numpy(esp_mix)
            dataset_per[:,int(esp_hops[i]):int(esp_hops[i]+esp_sizes[i])] = torch.from_numpy(esp_per)
            dataset_har[:,int(esp_hops[i]):int(esp_hops[i]+esp_sizes[i])] = torch.from_numpy(esp_har)

            if ((i+1)%(n_tracks/10))%10 == 0:
                print(str(((i+1)*100)/n_tracks)+'%')
    else:
        for i in range(n_tracks):
            x_mix = pd.read_csv('C:/Users/gusta/Misturas/Espectros/'+'Mixture_'+str(i+100)+'.csv')
            x_mix = x_mix.to_numpy()
            size_bytes = size_bytes + x_mix.nbytes
            esp_sizes[i] = x_mix.shape[1]
            if i > 0:
               esp_hops[i] = esp_hops[i-1]+esp_sizes[i-1]
        print("Dataset: "+str(dataset_type)+" Tamanho estimado: "+str(3*size_bytes/(1024**3))+" GB")

        dataset_mix = torch.zeros(int(np.floor((n_fft/2 +1))),int(esp_hops[n_tracks-1]+esp_sizes[n_tracks-1]))
        dataset_per = torch.zeros(int(np.floor((n_fft/2 +1))),int(esp_hops[n_tracks-1]+esp_sizes[n_tracks-1]))
        dataset_har = torch.zeros(int(np.floor((n_fft/2 +1))),int(esp_hops[n_tracks-1]+esp_sizes[n_tracks-1]))

        for i in range(n_tracks):
            esp_mix = pd.read_csv('C:/Users/gusta/Misturas/Espectros/'+'Mixture_'+str(i+100)+'.csv') 
            esp_mix = esp_mix.to_numpy()

            esp_per = pd.read_csv('C:/Users/gusta/Percurssao/Espectros/'+'Percurssao_'+str(i+100)+'.csv') 
            esp_per = esp_per.to_numpy()

            esp_har = pd.read_csv('C:/Users/gusta/Harmonico/Espectros/'+'Harmonico_'+str(i+100)+'.csv') 
            esp_har = esp_har.to_numpy()

            dataset_mix[:,int(esp_hops[i]):int(esp_hops[i]+esp_sizes[i])] = torch.from_numpy(esp_mix)
            dataset_per[:,int(esp_hops[i]):int(esp_hops[i]+esp_sizes[i])] = torch.from_numpy(esp_per)
            dataset_har[:,int(esp_hops[i]):int(esp_hops[i]+esp_sizes[i])] = torch.from_numpy(esp_har)

            if ((i+1)%(n_tracks/10))%10 == 0:
                print(str(((i+1)*100)/n_tracks)+'%')
    return esp_sizes, esp_hops, dataset_mix, dataset_per, dataset_har

#Function that returns total number of batches in the dataset
def total_batches(dataset_type, n_tracks, batch_size, size, stride):
    n_batches = 0
    
    if dataset_type == 'train':
        print("Dataset: "+str(dataset_type))
        for i in range(n_tracks):
            x_mix = pd.read_csv('C:/Users/gusta/Misturas/Espectros/'+'Mixture_'+str(i)+'.csv') 
            x_mix = x_mix.to_numpy()
            n_slices_vertical = np.floor((x_mix.shape[0] - size)/stride + 1)
            n_slices_horizontal = np.floor((x_mix.shape[1] - size)/stride + 1)
            n_slices_total = n_slices_vertical*n_slices_horizontal
            n_batches = n_batches + n_slices_total
            i = i +1
            if (i%(n_tracks/10))%10 == 0:
                print(str((i*100)/n_tracks)+'%')
        print("Banco de dados contém: ")
        print("Número de slices totais: "+str(n_batches)+", Número de Batches: "+str(np.floor(n_batches/batch_size))+", Tamanho: "+str(batch_size))
        print("Número de Slices restantes: "+str(n_batches%batch_size))
        return np.floor(n_batches/batch_size)
    else:
        print("Dataset: "+str(dataset_type))
        for i in range(n_tracks):
            x_mix = pd.read_csv('C:/Users/gusta/Misturas/Espectros/'+'Mixture_'+str(i+100)+'.csv') 
            x_mix = x_mix.to_numpy()
            n_slices_vertical = np.floor((x_mix.shape[0] - size)/stride + 1)
            n_slices_horizontal = np.floor((x_mix.shape[1] - size)/stride + 1)
            n_slices_total = n_slices_vertical*n_slices_horizontal
            n_batches = n_batches + n_slices_total
            i = i +1
            if (i%(n_tracks/10))%10 == 0:
                print(str((i*100)/n_tracks)+'%')
        print("Banco de dados contém: ")
        print("Número de slices totais: "+str(n_batches)+", Número de Batches: "+str(np.floor(n_batches/batch_size))+", Tamanho: "+str(batch_size))
        print("Número de Slices restantes: "+str(n_batches%batch_size))
        return np.floor(n_batches/batch_size)
    
#Function that returns batches (squares) for training
def fornece_batches(esp_sizes, esp_hops, dataset_mix, dataset_per, dataset_har, batch_size, current_track, slices_left, selection, size, stride, n_fft, normalization):
    if current_track == -1:
        current_track = selection[0]
        slices_left = 0
    
    n_slices_vertical = np.floor(((np.floor(n_fft/2 + 1) - size)/stride) + 1)    
    
    batch_mix = torch.zeros(int(batch_size),1,int(size),int(size))
    batch_per = torch.zeros(int(batch_size),1,int(size),int(size))
    batch_har = torch.zeros(int(batch_size),1,int(size),int(size))
    
    #stop_preparing = time.time()
    
    batch_index = 0
    while batch_index < batch_size:
        #start_loading = time.time()
        
        n_slices_horizontal = np.floor((esp_sizes[current_track] - size)/stride + 1)
        n_slices_total = n_slices_vertical*n_slices_horizontal

        if slices_left == 0: 
            slices_left = n_slices_total

        if slices_left > (batch_size-batch_index): 
            current_track_batch = batch_size - batch_index
        else: 
            current_track_batch = slices_left

        i = np.floor((n_slices_total-slices_left)/n_slices_vertical)
        j = (n_slices_total-slices_left)%n_slices_vertical

        slices_left = slices_left - current_track_batch

        #stop_definitions = time.time()

        #start_while = time.time()
        if normalization == True:
            while current_track_batch > 0:
                #print('Tamanho do espectrograma: '+str(dataset_mix.shape))
                #print('Limites Verticais: ('+str(j*stride)+", "+str(j*stride+size)+') , Limites Horizontais: ('+str(esp_hops[int(current_track)]+i*stride)+", "+str(esp_hops[int(current_track)]+i*stride+size)+")")

                batch_mix[int(batch_index),0,:,:] = dataset_mix[int(j*stride):int(j*stride+size),int(esp_hops[int(current_track)]+i*stride):int(esp_hops[int(current_track)]+i*stride+size)]
                #print("Mix Mean",batch_mix[int(batch_index),0,:,:].mean(),"Mix Var",batch_mix[int(batch_index),0,:,:].var())
                batch_mix[int(batch_index),0,:,:] = (batch_mix[int(batch_index),0,:,:]-batch_mix[int(batch_index),0,:,:].mean())/(torch.sqrt(batch_mix[int(batch_index),0,:,:].var()+1e-16))
                batch_per[int(batch_index),0,:,:] = dataset_per[int(j*stride):int(j*stride+size),int(esp_hops[int(current_track)]+i*stride):int(esp_hops[int(current_track)]+i*stride+size)] 
                #print("Per Mean",batch_per[int(batch_index),0,:,:].mean(),"Per Var",batch_per[int(batch_index),0,:,:].var())
                batch_per[int(batch_index),0,:,:] = (batch_per[int(batch_index),0,:,:]-batch_per[int(batch_index),0,:,:].mean())/(torch.sqrt(batch_per[int(batch_index),0,:,:].var()+1e-16))
                batch_har[int(batch_index),0,:,:] = dataset_har[int(j*stride):int(j*stride+size),int(esp_hops[int(current_track)]+i*stride):int(esp_hops[int(current_track)]+i*stride+size)]
                #print("Har Mean",batch_har[int(batch_index),0,:,:].mean(),"Har Var",batch_har[int(batch_index),0,:,:].var())
                batch_har[int(batch_index),0,:,:] = (batch_har[int(batch_index),0,:,:]-batch_har[int(batch_index),0,:,:].mean())/(torch.sqrt(batch_har[int(batch_index),0,:,:].var()+1e-16))
                
                j = (j+1)%n_slices_vertical
                if j == 0: i = i + 1
                batch_index = batch_index + 1
                current_track_batch = current_track_batch - 1
        else:
            while current_track_batch > 0:
                    #print('Tamanho do espectrograma: '+str(dataset_mix.shape))
                    #print('Limites Verticais: ('+str(j*stride)+", "+str(j*stride+size)+') , Limites Horizontais: ('+str(esp_hops[int(current_track)]+i*stride)+", "+str(esp_hops[int(current_track)]+i*stride+size)+")")

                    batch_mix[int(batch_index),0,:,:] = dataset_mix[int(j*stride):int(j*stride+size),int(esp_hops[int(current_track)]+i*stride):int(esp_hops[int(current_track)]+i*stride+size)]
                    batch_per[int(batch_index),0,:,:] = dataset_per[int(j*stride):int(j*stride+size),int(esp_hops[int(current_track)]+i*stride):int(esp_hops[int(current_track)]+i*stride+size)] 
                    batch_har[int(batch_index),0,:,:] = dataset_har[int(j*stride):int(j*stride+size),int(esp_hops[int(current_track)]+i*stride):int(esp_hops[int(current_track)]+i*stride+size)]

                    j = (j+1)%n_slices_vertical
                    if j == 0: i = i + 1
                    batch_index = batch_index + 1
                    current_track_batch = current_track_batch - 1

        #print("Track:"+str(current_track)+" Faltantes:"+str(slices_left)+" Slices Totais:"+str(n_slices_total)+" Batch Size"+"("+str(batch_index)+")")           
        if slices_left == 0: 
            current_track = selection[(np.where(selection==int(current_track)))[0].item()+1]
            
        #print('----------------------------')   
        
    return current_track, slices_left, batch_mix, batch_per, batch_har 

#Apply model to obtain stems
def apply_model(indice_musica, model, batch_size, n_window, n_fft, hop_length, size, stride, normalization, pct_limiar): 
    
    model.eval()
    
    with torch.no_grad():
        track = pd.read_csv('C:/Users/gusta/Misturas/'+'Mixture_'+str(indice_musica)+'.csv')
        track = track.to_numpy()
        track = track.T

        track_per = pd.read_csv('C:/Users/gusta/Percurssao/'+'Percurssao_'+str(indice_musica)+'.csv')
        track_per = track_per.to_numpy()
        track_per = track_per.T

        track_har = pd.read_csv('C:/Users/gusta/Harmonico/'+'Harmonico_'+str(indice_musica)+'.csv')
        track_har = track_har.to_numpy()
        track_har = track_har.T

        esp = (librosa.stft(track[0], win_length = n_window, n_fft = n_fft, hop_length = hop_length, window = 'hamming', center=False))       
        
        x_mix = pd.read_csv('C:/Users/gusta/Misturas/Espectros/'+'Mixture_'+str(indice_musica)+'.csv')
        x_mix = x_mix.to_numpy()

        n_slices_vertical = np.floor(((np.floor(n_fft/2 + 1) - size)/stride) + 1)  
        n_slices_horizontal = np.floor((x_mix.shape[1] - size)/stride + 1)
        n_slices_total = n_slices_vertical*n_slices_horizontal
        
        #print("Dim_Vertical", x_mix.shape[0], "Dim_Horizontal", x_mix.shape[1])
        sobra_vertical = x_mix.shape[0]-(size+(n_slices_vertical-1)*stride)
        sobra_horizontal = x_mix.shape[1]-(size+(n_slices_horizontal-1)*stride)
        #print("Sobra_Vertical", sobra_vertical, "Sobra_Horizontal", sobra_horizontal)
        
        if sobra_vertical > 0:
            enxerto_vertical = np.zeros((int(sobra_vertical),int(x_mix.shape[1])))
            enxerto_vertical = esp[int(x_mix.shape[0]-sobra_vertical):int(x_mix.shape[0]),0:int(x_mix.shape[1]-sobra_horizontal)]

        phase = np.angle(esp[0:int(x_mix.shape[0]-sobra_vertical),0:int(x_mix.shape[1]-sobra_horizontal)])
        
        x_per = np.zeros((int(x_mix.shape[0]-sobra_vertical),int(x_mix.shape[1]-sobra_horizontal)))
        x_har = np.zeros((int(x_mix.shape[0]-sobra_vertical),int(x_mix.shape[1]-sobra_horizontal)))
        batch_mix = torch.zeros(int(n_slices_total),1,int(size),int(size))
        batch_per = torch.zeros(int(n_slices_total),1,int(size),int(size))
        batch_har = torch.zeros(int(n_slices_total),1,int(size),int(size))

        x = 0
        y = 0
        batch_index = 0

        while batch_index < n_slices_total:
                batch_mix[int(batch_index),0,:,:] = torch.from_numpy(x_mix[int(y*stride):int(y*stride+size),int(x*stride):int(x*stride+size)])
                if normalization == True:
                    batch_mix[int(batch_index),0,:,:] = (batch_mix[int(batch_index),0,:,:]-batch_mix[int(batch_index),0,:,:].mean())/(torch.sqrt(batch_mix[int(batch_index),0,:,:].var()+1e-16))
                
                y = (y+1)%n_slices_vertical
                if y == 0: x = x + 1
                batch_index = batch_index + 1

        x = np.floor(n_slices_total/batch_size)
        y = np.floor(n_slices_total%batch_size)
        batch_index = 0
        
        batch_mix = batch_mix.to(device)
        for batch_index in range(int(x)):
            batch_per[int(batch_index*batch_size):int(batch_index*batch_size+batch_size),:,:,:], batch_har[int(batch_index*batch_size):int(batch_index*batch_size+batch_size),:,:,:] = model(batch_mix[int(batch_index*batch_size):int(batch_index*batch_size+batch_size),:,:,:])
        
        batch_per[int(x*batch_size):int(x*batch_size+y),:,:,:], batch_har[int(x*batch_size):int(x*batch_size+y),:,:,:] = model(batch_mix[int(x*batch_size):int(x*batch_size+y),:,:,:])
        
        batch_per = batch_per.to('cpu')
        batch_har = batch_har.to('cpu')
        

        
        x = 0
        y = 0
        batch_index = 0

        while batch_index < n_slices_total:
            if batch_index < n_slices_vertical:
                if y == 0:
                    x_per[int(y*stride):int(y*stride+size),int(x*stride):int(x*stride+size)] = batch_per[int(batch_index),0,:,:]
                    x_har[int(y*stride):int(y*stride+size),int(x*stride):int(x*stride+size)] = batch_har[int(batch_index),0,:,:]
                else:
                    x_per[int(y*stride+size-stride):int(y*stride+size),int(x*stride):int(x*stride+size)] = batch_per[int(batch_index),0,int(size-stride):int(size),:]
                    x_har[int(y*stride+size-stride):int(y*stride+size),int(x*stride):int(x*stride+size)] = batch_har[int(batch_index),0,int(size-stride):int(size),:]
            else:
                if y == 0:
                    x_per[int(y*stride):int(y*stride+size),int(x*stride+size-stride):int(x*stride+size)] = batch_per[int(batch_index),0,:,int(size-stride):int(size)]
                    x_har[int(y*stride):int(y*stride+size),int(x*stride+size-stride):int(x*stride+size)] = batch_har[int(batch_index),0,:,int(size-stride):int(size)]
                else:
                    x_per[int(y*stride+size-stride):int(y*stride+size),int(x*stride+size-stride):int(x*stride+size)] = batch_per[int(batch_index),0,int(size-stride):int(size),int(size-stride):int(size)]
                    x_har[int(y*stride+size-stride):int(y*stride+size),int(x*stride+size-stride):int(x*stride+size)] = batch_har[int(batch_index),0,int(size-stride):int(size),int(size-stride):int(size)]
            
            y = (y+1)%n_slices_vertical
            if y == 0: x = x + 1
            batch_index = batch_index + 1
        
  
        #Máscara 
        
        mask_per = x_per/(x_har+x_per+1e-16)
        mask_har = x_har/(x_har+x_per+1e-16)

        limiar_mask_per = mask_per.mean()*pct_limiar
        limiar_mask_har = mask_har.mean()*pct_limiar
        #print("limiar_per",limiar_mask_per,"limiar_har",limiar_mask_har)

        for i in range(int(x_mix.shape[0]-sobra_vertical)):
            for j in range(int(x_mix.shape[1]-sobra_horizontal)):
                if mask_per[i,j] < limiar_mask_per:
                    mask_per[i,j] = 0
                if mask_har[i,j] < limiar_mask_har:
                    mask_har[i,j] = 0
        
                    
        enxerto_vertical_per_mask = x_mix[int(x_mix.shape[0]-sobra_vertical-1),0:int(x_mix.shape[1]-sobra_horizontal)]*mask_per
        enxerto_vertical_har_mask = x_mix[int(x_mix.shape[0]-sobra_vertical-1),0:int(x_mix.shape[1]-sobra_horizontal)]*mask_har
        
        x_mix_mask = x_mix[0:int(x_mix.shape[0]-sobra_vertical),0:int(x_mix.shape[1]-sobra_horizontal)]
        
        x_per_mask = x_mix_mask*mask_per
        x_har_mask = x_mix_mask*mask_har
        
        #Return phase and cut slice
        x_per_mask = x_per_mask*np.cos(phase)+1j*(x_per_mask*np.sin(phase))
        x_har_mask = x_har_mask*np.cos(phase)+1j*(x_har_mask*np.sin(phase))
        
        enxerto_vertical_per_mask = enxerto_vertical_per_mask*np.cos(np.angle(enxerto_vertical))+1j*(enxerto_vertical_per_mask*np.sin(np.angle(enxerto_vertical)))
        enxerto_vertical_har_mask = enxerto_vertical_har_mask*np.cos(np.angle(enxerto_vertical))+1j*(enxerto_vertical_har_mask*np.sin(np.angle(enxerto_vertical)))
        
        if sobra_vertical > 0:
            x_per_mask = np.vstack((x_per_mask,enxerto_vertical_per_mask))
            x_har_mask = np.vstack((x_har_mask,enxerto_vertical_har_mask))
        
        #IFFT

        est_per = librosa.istft(x_per_mask, win_length = n_window, n_fft = n_fft, hop_length = hop_length, window = 'hamming', center=False)
        est_har = librosa.istft(x_har_mask, win_length = n_window, n_fft = n_fft, hop_length = hop_length, window = 'hamming', center=False)
    
    #Para comparação, deve-se retirar a parte cortada
    track = track[:,0:int(est_per.size)]
    track_per = track_per[:,0:int(est_per.size)]
    track_har = track_har[:,0:int(est_har.size)]
    
    return mask_per, mask_har, track, track_per, track_har, est_per, est_har

In [ ]:
#Visualização de espectrogramas a partir das Stems diretamente
music_index =  100
stem = 3
sr = 16000
hop_length = 256
n_window = 1024
n_fft = 1024
stem_espectrograma(music_index, stem, sr, hop_length, n_window, n_fft)

In [ ]:
#Visualização de espectrogramas salvos
track = 45
print("Mistura")
mostra_espectrograma(0,track, 256, 1024,1024)
print("Percussivo")
mostra_espectrograma(1,track, 256, 1024,1024)
print("Harmônico")
mostra_espectrograma(2,track, 256, 1024,1024)

In [ ]:
# Preparação do dataset em espectrogramas
# Rodar apenas se não tiver os espectrogramas salvos ainda
hop_length = 256
n_fft=1024
n_window=1024
prepara_dataset(hop_length, n_window, n_fft)

In [ ]:
#Modelo
#Net model
class ConvAutoEncoder(nn.Module):
    def __init__(self):
        super(ConvAutoEncoder, self).__init__()
        
        #Redução de dimensão
        self.encoder = nn.Sequential(nn.Conv2d(1,64, stride = 1, padding =1, kernel_size=(3,3)),
                                     nn.ReLU(),
                                     nn.MaxPool2d(2, 2),
                                     nn.Conv2d(64, 32, stride = 1, padding =1, kernel_size=(3,3)),
                                     nn.ReLU(),
                                     nn.MaxPool2d(2, 2),
                                     nn.Conv2d(32, 16, stride = 1, padding =1, kernel_size=(3,3)),
                                     nn.ReLU(),
                                     nn.MaxPool2d(2, 2))
    
    
        #Aumento de dimensao para parcela percussiva
        self.decoder_per = nn.Sequential(nn.ConvTranspose2d(16, 16, stride = 1, padding =1, kernel_size=(3,3)),
                                         nn.ReLU(),
                                         nn.UpsamplingNearest2d(scale_factor=2),
                                         nn.ConvTranspose2d(16, 32, stride = 1, padding =1, kernel_size=(3,3)),
                                         nn.ReLU(),
                                         nn.UpsamplingNearest2d(scale_factor=2),
                                         nn.ConvTranspose2d(32, 64, stride = 1, padding =1, kernel_size=(3,3)),
                                         nn.ReLU(),
                                         nn.UpsamplingNearest2d(scale_factor=2),
                                         nn.ConvTranspose2d(64, 1, stride = 1, padding =1, kernel_size=(3,3)),
                                         nn.ReLU())
     
        
        #Aumento de dimensao para parcela harmônica
        self.decoder_har = nn.Sequential(nn.ConvTranspose2d(16, 16, stride = 1, padding =1, kernel_size=(3,3)),
                                         nn.ReLU(),
                                         nn.UpsamplingNearest2d(scale_factor=2),
                                         nn.ConvTranspose2d(16, 32, stride = 1, padding =1, kernel_size=(3,3)),
                                         nn.ReLU(),
                                         nn.UpsamplingNearest2d(scale_factor=2),
                                         nn.ConvTranspose2d(32, 64, stride = 1, padding =1, kernel_size=(3,3)),
                                         nn.ReLU(),
                                         nn.UpsamplingNearest2d(scale_factor=2),
                                         nn.ConvTranspose2d(64, 1, stride = 1, padding =1, kernel_size=(3,3)),
                                         nn.ReLU())
        
    def forward(self, x):
        x = self.encoder(x)  
        x_per = self.decoder_per(x)  
        x_har = self.decoder_har(x)    
        
        return x_per, x_har

In [ ]:
#Parametros
n_tracks_train = 100
n_tracks_validation = 50
hop_length = 256
n_fft=1024
n_window=1024
size=512
stride=128
batch_size = 5 #suportado pela GPU (Variável com as configurações)

In [ ]:
# Hyper-parametros 
num_epochs = 20
learning_rate = 0.1
n_batch_train = total_batches('train', n_tracks_train, batch_size, size, stride)
print()
n_batch_validation = total_batches('validation', n_tracks_validation, batch_size, size, stride)
normalization = False #Normlização do Batch

In [ ]:
learning_rate = 0.1

In [ ]:
#Gpu support

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    
model = ConvAutoEncoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for name, param in model.named_parameters():
    print(name, '\t\t\t', param.shape,'\t\t\t',param.device)

In [ ]:
#Loading dataset
esp_sizes_train, esp_hops_train, dataset_mix_train, dataset_per_train, dataset_har_train = carrega_dataset('train', n_tracks_train, hop_length, n_window, n_fft)
esp_sizes_validation, esp_hops_validation, dataset_mix_validation, dataset_per_validation, dataset_har_validation = carrega_dataset('validation', n_tracks_validation, hop_length, n_window, n_fft)

In [ ]:
#Modelo do zero
loaded = 0
normalization = True

In [ ]:
#Load model and error, if saved
loaded = 1
model.load_state_dict(torch.load('C:/Users/gusta/Modelo_1.2_50epoch_rate=0.1'))

epoch_loss_train = {}
epoch_loss_validation = {}

epoch_loss_train_df = pd.read_csv('C:/Users/gusta/Loss_train_1.2_50epoch_rate=0.1')
epoch_loss_validation_df = pd.read_csv('C:/Users/gusta/Loss_validation_1.2_50epoch_rate=0.1')
 
epoch_loss_train['total'] = epoch_loss_train_df['total'].values.tolist()
epoch_loss_train['percursive'] = epoch_loss_train_df['percursive'].values.tolist()
epoch_loss_train['harmonic'] = epoch_loss_train_df['percursive'].values.tolist()

epoch_loss_validation['total'] = epoch_loss_validation_df['total'].values.tolist()
epoch_loss_validation['percursive'] = epoch_loss_validation_df['percursive'].values.tolist()
epoch_loss_validation['harmonic'] = epoch_loss_validation_df['percursive'].values.tolist()

In [ ]:
#Training

#Caso comece do zero
if loaded == 0:
    epoch_loss_train = {}
    epoch_loss_train['total'] = []
    epoch_loss_train['percursive'] = []
    epoch_loss_train['harmonic'] = []
    
    epoch_loss_validation = {}
    epoch_loss_validation['total'] = []
    epoch_loss_validation['percursive'] = []
    epoch_loss_validation['harmonic'] = []
    

start_time = time.time()
for epoch in range(int(num_epochs)):
    model.train()
    selection_train = np.random.choice(n_tracks_train, size=n_tracks_train, replace=False)
    
    running_loss = 0
    running_loss_per = 0
    running_loss_har = 0
    
    current_track = -1
    slices_left = 0
    
    print()
    print("Train ")
    for i in range(int(n_batch_train)):
        #Obtém Batch
        current_track, slices_left, x_mix, x_per, x_har = fornece_batches(esp_sizes_train, esp_hops_train, dataset_mix_train, dataset_per_train, dataset_har_train, batch_size, current_track, slices_left, selection_train, size, stride, n_fft, normalization = normalization)

        #Treino na GPU
        #x_mix = x_mix.to(device) 
        x_per = x_per.to(device)
        x_har = x_har.to(device)
        
        # Forward pass and error
        optimizer.zero_grad()
        #x_per_est, x_har_est = model(x_mix)
        x_per_est, x_har_est = model(x_per + x_har)
        loss_per = criterion(x_per_est, x_per)
        loss_har = criterion(x_har_est, x_har)
        loss = criterion(x_har_est, x_har)+criterion(x_per_est, x_per)
        
        running_loss = running_loss + loss.item()
        running_loss_per = running_loss_per + loss_per.item()
        running_loss_har = running_loss_har + loss_har.item()
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print (f'Train mode : Current Track [{current_track}], Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_batch_train}], Duration Time: {time.time()-start_time:.4f}s, Loss: {loss.item():.4f}, Loss Per: {loss_per.item():.4f}, Loss Har: {loss_har.item():.4f}')

    
    epoch_loss_train['total'].append(running_loss/n_batch_train)
    epoch_loss_train['percursive'].append(running_loss_per/n_batch_train)
    epoch_loss_train['harmonic'].append(running_loss_har/n_batch_train)
    
    model.eval()
    with torch.no_grad():
        selection_validation = np.random.choice(n_tracks_validation, size=n_tracks_validation, replace=False)

        running_loss = 0
        running_loss_per = 0
        running_loss_har = 0

        current_track = -1
        slices_left = 0

        print()
        print("Validation ")

        for i in range(int(n_batch_validation)):
            #Obtém Batch
            current_track, slices_left, x_mix, x_per, x_har = fornece_batches(esp_sizes_validation, esp_hops_validation, dataset_mix_validation, dataset_per_validation, dataset_har_validation, batch_size, current_track, slices_left, selection_validation, size, stride, n_fft, normalization = normalization)

            #Validação na GPU
            #x_mix = x_mix.to(device)
            x_per = x_per.to(device)
            x_har = x_har.to(device)

            #Model output and error
            #x_per_est, x_har_est = model(x_mix)
            x_per_est, x_har_est = model(x_per + x_har)
            loss_per = criterion(x_per_est, x_per)
            loss_har = criterion(x_har_est, x_har)
            loss = criterion(x_har_est, x_har)+criterion(x_per_est, x_per)

            running_loss = running_loss + loss.item()
            running_loss_per = running_loss_per + loss_per.item()
            running_loss_har = running_loss_har + loss_har.item()


            if (i+1) % 10 == 0:
                print (f'Validation mode : Current Track [{current_track}], Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_batch_validation}] , Duration Time: {time.time()-start_time:.4f}s, Loss: {loss.item():.4f}, Loss Per: {loss_per.item():.4f}, Loss Har: {loss_har.item():.4f}')

        epoch_loss_validation['total'].append(running_loss/n_batch_validation)
        epoch_loss_validation['percursive'].append(running_loss_per/n_batch_validation)
        epoch_loss_validation['harmonic'].append(running_loss_har/n_batch_validation)

    print("----------------------------------------------------------")
    
plt.figure(1)
plt.title('Mean Loss per epoch')
plt.plot(epoch_loss_train['total'])
plt.plot(epoch_loss_train['percursive'])
plt.plot(epoch_loss_train['harmonic'])
plt.plot(epoch_loss_validation['total'])
plt.plot(epoch_loss_validation['percursive'])
plt.plot(epoch_loss_validation['harmonic'])
plt.legend(['Train Total','Train Percursiva','Train Harmonica','Val Total','Val Percursiva','Val Harmonica'])

In [ ]:
plt.figure(1)
plt.plot(repoch_loss_train['total'])
plt.plot(epoch_loss_validation['total'])

plt.legend(['Treino','Validação'])

In [ ]:
#FIR para diminuir a variância da validação
for i in range(50):
    if i>2 and i<47:
        epoch_loss_validation['total'][i] = epoch_loss_validation['total'][i]*(4/16)+epoch_loss_validation['total'][i-1]*(3/16)+epoch_loss_validation['total'][i-2]*(2/16)+epoch_loss_validation['total'][i-3]*(1/16)+epoch_loss_validation['total'][i+1]*(3/16)+epoch_loss_validation['total'][i+2]*(2/16)+epoch_loss_validation['total'][i+3]*(1/16)

In [ ]:
#Verificar Modelo
print(model.state_dict())

In [ ]:
#Save Model and error 
torch.save(model.state_dict(), 'C:/Users/gusta/Modelo_1.1_20epoch_rate=0.1')

epoch_loss_train_df = pd.DataFrame(epoch_loss_train)
epoch_loss_train_df.to_csv('C:/Users/gusta/Loss_train_1.1_20epoch_rate=0.1', index = False)

epoch_loss_validation_df = pd.DataFrame(epoch_loss_validation)
epoch_loss_validation_df.to_csv('C:/Users/gusta/Loss_validation_1.1_20epoch_rate=0.1', index = False)

In [ ]:
#Clean GPU memory
optimizer.zero_grad()
del x_mix
del x_per
del x_har
del x_har_est
del x_per_est
del loss
del optimizer
del model
gc.collect()
torch.cuda.empty_cache()
print(torch.cuda.get_device_name(0))
print('Memory Usage:')
print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,5), 'GB')
print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,5), 'GB')

In [ ]:
#Testing
index = 139
normalization = True
pct_limiar = 0
mask_per, mask_har, track, track_per, track_har, est_per, est_har = apply_model(index, model, batch_size, n_window, n_fft, hop_length, size, stride, normalization = normalization, pct_limiar = pct_limiar)

print("Track: "+str(index)+ " Stem: Mixture")
ipd.Audio(track, rate = 16000)

In [ ]:
plt.imshow(mask_per,aspect='auto',vmin=0, origin = 'lower')

In [ ]:
plt.imshow(mask_har, aspect='auto',vmin=0, origin = 'lower')

In [ ]:
print("Track: "+str(index)+" Stem: Percursive")
ipd.Audio(track_per, rate = 16000)

In [ ]:
print("Track: "+str(index)+" Stem: Harmonic")
ipd.Audio(track_har, rate = 16000)

In [ ]:
print("Model output with Mask type 1: "+str(index)+" Stem: Percursive")
ipd.Audio(est_per, rate = 16000)

In [ ]:
print("Model output with Mask type 1: "+str(index)+" Stem: Harmonic")
ipd.Audio(est_har, rate = 16000)

In [ ]:
#Validation

normalization = True
n_tracks_train = 100
n_tracks_validation = 50
hop_length = 256
n_fft=1024
n_window=1024
size=512
stride=128
pct_limiar = 0

metric_train = {}
metric_train['Per SDR'] = []
metric_train['Per SAR'] = []
metric_train['Per SIR'] = []
metric_train['Har SDR'] = []
metric_train['Har SAR'] = []
metric_train['Har SIR'] = []


metric_validation = {}
metric_validation['Per SDR'] = []
metric_validation['Per SAR'] = []
metric_validation['Per SIR'] = []
metric_validation['Har SDR'] = []
metric_validation['Har SAR'] = []
metric_validation['Har SIR'] = []

sdr = 0
sar = 0
sir = 0
isr = 0
perm = 0
running_sdr_per =0
running_sar_per =0
running_sir_per =0
running_sdr_har =0
running_sar_har =0
running_sir_har =0


start_time = time.time()
print()
print("Train")

for i in range(int(n_tracks_train)):

    mask_per, mask_har, track, track_per, track_har, est_per, est_har = apply_model(i, model, batch_size, n_window, n_fft, hop_length, size, stride, normalization = normalization, pct_limiar = pct_limiar)
    #reference = np.array([track_per.reshape(-1,1),track_har.reshape(-1,1)]) #harmonic reference is just harmonic target
    reference = np.array([track_per.reshape(-1,1),(track-track_per).reshape(-1,1)]) #harmonic reference is everything else drums
    estimated = np.array([est_per.reshape(-1,1),est_har.reshape(-1,1)])

    sdr,isr,sir,sar,perm = museval.metrics.bss_eval(reference, estimated, window = np.inf)
    running_sdr_per = running_sdr_per + 10**(sdr[0].mean()/10)
    running_sar_per = running_sar_per + 10**(sar[0].mean()/10)
    running_sir_per = running_sir_per + 10**(sir[0].mean()/10) 
    running_sdr_har = running_sdr_har + 10**(sdr[1].mean()/10)
    running_sar_har = running_sar_har + 10**(sar[1].mean()/10)
    running_sir_har = running_sir_har + 10**(sir[1].mean()/10) 
    
    if (i+1) % 10 == 0:
        print (f'Train mode : Current Track [{i}], Duration Time: {time.time()-start_time:.4f}s')

metric_train['Per SDR'].append(10*np.log10(running_sdr_per/n_tracks_train+1e-16))
metric_train['Per SAR'].append(10*np.log10(running_sar_per/n_tracks_train+1e-16))
metric_train['Per SIR'].append(10*np.log10(running_sir_per/n_tracks_train+1e-16)) 

metric_train['Har SDR'].append(10*np.log10(running_sdr_har/n_tracks_train+1e-16))
metric_train['Har SAR'].append(10*np.log10(running_sar_har/n_tracks_train+1e-16))
metric_train['Har SIR'].append(10*np.log10(running_sir_har/n_tracks_train+1e-16)) 

print("----------------------------------------------------------")

running_sdr_per =0
running_sar_per =0
running_sir_per =0
running_sdr_har =0
running_sar_har =0
running_sir_har =0

print()
print("Validation")
for i in range(int(n_tracks_validation)):

    mask_per, mask_har, track, track_per, track_har, est_per, est_har = apply_model(i+100, model, batch_size, n_window, n_fft, hop_length, size, stride, normalization = normalization, pct_limiar = pct_limiar)
    #reference = np.array([track_per.reshape(-1,1),track_har.reshape(-1,1)]) #harmonic reference is just harmonic target
    reference = np.array([track_per.reshape(-1,1),(track-track_per).reshape(-1,1)]) #harmonic reference is everything else drums
    estimated = np.array([est_per.reshape(-1,1),est_har.reshape(-1,1)])
    
    sdr,isr,sir,sar,perm = museval.metrics.bss_eval(reference, estimated, window = np.inf)
    running_sdr_per = running_sdr_per + 10**(sdr[0].mean()/10)
    running_sar_per = running_sar_per + 10**(sar[0].mean()/10)
    running_sir_per = running_sir_per + 10**(sir[0].mean()/10) 
    running_sdr_har = running_sdr_har + 10**(sdr[1].mean()/10)
    running_sar_har = running_sar_har + 10**(sar[1].mean()/10)
    running_sir_har = running_sir_har + 10**(sir[1].mean()/10) 
    

    if (i+1) % 10 == 0:
        print (f'Validation mode : Current Track [{i}], Duration Time: {time.time()-start_time:.4f}s')

metric_validation['Per SDR'].append(10*np.log10(running_sdr_per/n_tracks_validation+1e-16))
metric_validation['Per SAR'].append(10*np.log10(running_sar_per/n_tracks_validation+1e-16))
metric_validation['Per SIR'].append(10*np.log10(running_sir_per/n_tracks_validation+1e-16)) 

metric_validation['Har SDR'].append(10*np.log10(running_sdr_har/n_tracks_validation+1e-16))
metric_validation['Har SAR'].append(10*np.log10(running_sar_har/n_tracks_validation+1e-16))
metric_validation['Har SIR'].append(10*np.log10(running_sir_har/n_tracks_validation+1e-16)) 


In [ ]:
#Save metrics
metric_train_df = pd.DataFrame(metric_train)
metric_train_df.to_csv('C:/Users/gusta/Loss_train_1.1_50epoch_limiar=corr', index = False)

metric_validation_df = pd.DataFrame(metric_validation)
metric_validation_df.to_csv('C:/Users/gusta/Loss_validation_1.1_50epoch_limiar=corr', index = False)

In [ ]:
#Load metrics
metric_train = {}
metric_validation = {}

metric_train_df = pd.read_csv('C:/Users/gusta/Loss_train_1.1_50epoch_limiar=100')
metric_validation_df = pd.read_csv('C:/Users/gusta/Loss_validation_1.1_50epoch_limiar=100')
 
metric_train['Per SDR'] = metric_train_df['Per SDR'].values.tolist()
metric_train['Per SAR'] = metric_train_df['Per SAR'].values.tolist()
metric_train['Har SDR'] = metric_train_df['Har SDR'].values.tolist()
metric_train['Har SAR'] = metric_train_df['Har SAR'].values.tolist()

metric_validation['Per SDR'] = metric_validation_df['Per SDR'].values.tolist()
metric_validation['Per SAR'] = metric_validation_df['Per SAR'].values.tolist()
metric_validation['Har SDR'] = metric_validation_df['Har SDR'].values.tolist()
metric_validation['Har SAR'] = metric_validation_df['Har SAR'].values.tolist()

In [ ]:
#Show metrics
metric_validation = {}
metric_validation_df = pd.read_csv('C:/Users/gusta/Loss_validation_1.1_50epoch_limiar=0')
metric_validation_norm_df = pd.read_csv('C:/Users/gusta/Loss_validation_1.2_50epoch_limiar=0')
data = [[float("%.2f" % metric_validation_df['Per SDR'][0]),float("%.2f" % metric_validation_norm_df['Per SDR'][0]),2.51],[float("%.2f" % metric_validation_df['Per SAR'][0]),float("%.2f" % metric_validation_norm_df['Per SAR'][0]),3.53],[float("%.2f" % metric_validation_df['Per SIR'][0]),float("%.2f" % metric_validation_norm_df['Per SIR'][0]),14.15]]

X = np.arange(3)
fig = plt.figure(1)
ax = plt.bar(X + 0.00, data[0], color = 'b', width = 0.25)
ax1 = plt.bar(X + 0.25, data[1], color = 'g', width = 0.25,tick_label = ['Sem Norm','Norm','Ref'])
ax2 = plt.bar(X + 0.50, data[2], color = 'r', width = 0.25)
plt.legend(['SDR','SAR','SIR'])
plt.bar_label(ax)
plt.bar_label(ax1)
plt.bar_label(ax2)

metric_validation = {}
metric_validation_df = pd.read_csv('C:/Users/gusta/Loss_validation_1.1_50epoch_limiar=0')
metric_validation_norm_df = pd.read_csv('C:/Users/gusta/Loss_validation_1.2_50epoch_limiar=0')
data = [[float("%.2f" % metric_validation_df['Har SDR'][0]),float("%.2f" % metric_validation_norm_df['Har SDR'][0]),9.61],[float("%.2f" % metric_validation_df['Har SAR'][0]),float("%.2f" % metric_validation_norm_df['Har SAR'][0]),10.20],[float("%.2f" % metric_validation_df['Har SIR'][0]),float("%.2f" % metric_validation_norm_df['Har SIR'][0]),20.27]]

X = np.arange(3)
fig = plt.figure(2)
ax = plt.bar(X + 0.00, data[0], color = 'b', width = 0.25)
ax1 = plt.bar(X + 0.25, data[1], color = 'g', width = 0.25,tick_label = ['Sem Norm','Norm','Ref'])
ax2 = plt.bar(X + 0.50, data[2], color = 'r', width = 0.25)
plt.legend(['SDR','SAR','SIR'])
plt.bar_label(ax)
plt.bar_label(ax1)
plt.bar_label(ax2)

In [ ]:
plt.figure(4)
plt.title('Percusive')
ax = plt.bar('SDR',metric_validation['Per SDR'])
ax1 = plt.bar('SAR',metric_validation['Per SAR'])
ax2 = plt.bar('SIR',metric_validation['Per SIR'])
plt.legend(['SDR','SAR','SIR'])
plt.bar_label(ax)
plt.bar_label(ax1)
plt.bar_label(ax2)